In [ ]:
#| default_exp ea

# EA SPD 

> Connect and interface with the public online files

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import pandas as pd
import numpy as np
import httpx
import json
import calendar
import datetime
import lxml
import asyncio
import re
from zipfile import ZipFile
from io import BytesIO
from io import StringIO

Call the EA website based on `year` to get available files

In [ ]:
#|export
def _date(x):
    ''' Parse the filename to return file date '''
    tmp = re.search(r'(\d{1,2})111(\d{4})(\d{2})', x).groups()
    return f'{tmp[0]}/{tmp[2]}/{tmp[1]}'    

def extract_spd_list(year): 
    ''' Get list of available downloads from the casefiles '''
    return (pd.read_html(f'https://www.emi.ea.govt.nz/Wholesale/Datasets/FinalPricing/CaseFiles/{year}')[0] 
            .assign(file_date = lambda df_: pd.to_datetime(df_.Name.apply(_date), dayfirst=True)+ pd.DateOffset(days=1),
                    file_size = lambda df_: (df_['File size'].str.replace(',', '').str[:-3]).astype(int))
            .pipe(lambda df_: df_.loc[df_.file_size > 1000].drop_duplicates(subset=['file_date']))
            .set_index('file_date')
            .sort_index()
            )

In [ ]:
extract_spd_list(2022)

,Name,Date modified,File size,file_size
file_date,,,,
2022-01-01,MSS_311112021121100996_0X.ZIP,01 Jan 2022,"6,380 KB",6380
2022-01-02,MSS_11112022011100274_0X.ZIP,04 Jan 2022,"6,426 KB",6426
2022-01-03,MSS_21112022011100481_0X.ZIP,03 Jan 2022,"6,457 KB",6457
2022-01-04,MSS_31112022011100206_0X.ZIP,04 Jan 2022,"6,458 KB",6458
2022-01-05,MSS_41112022011100952_0X.ZIP,05 Jan 2022,"6,453 KB",6453
...,...,...,...,...
2022-08-06,MSS_51112022081200827_0X.ZIP,07 Aug 2022,"6,100 KB",6100
2022-08-07,MSS_61112022081200811_0X.ZIP,07 Aug 2022,"6,083 KB",6083
2022-08-08,MSS_71112022081200614_0X.ZIP,08 Aug 2022,"6,165 KB",6165


In [ ]:
#|export
def _branch(file):
    ''' Get raw spd files and return as dataframe for branches '''
    df = pd.concat((pd.DataFrame(_extract(file, fn, 'SOLUTION,BRANCH')[1:], 
                                    columns = _extract(file, fn, 'SOLUTION,BRANCH')[0])
                        for fn in file.namelist() if '.SPDSOLVED' in fn[-10:]), axis=0)
    cols = ['INTERVAL', 'BRANCHNAME', 'FROM_STATION', 'TO_STATION', 'FROM_MW', 'TO_MW', 'BRANCHLOSSES']
    return (df
        [cols]
        .assign(INTERVAL = lambda df_: pd.to_datetime(df_.INTERVAL.replace({';':':'}, regex=True),  dayfirst = True),
                TO_STATION = lambda df_: df_.TO_STATION.str.strip(),
                FROM_STATION = lambda df_: df_.FROM_STATION.str.strip(),
                FROM_MW = lambda df_: df_.FROM_MW.astype('float32'),
                TO_MW = lambda df_: df_.TO_MW.astype('float32'),
                BRANCHLOSSES = lambda df_: df_.BRANCHLOSSES.astype('float32'),
                ))

def _bus(file):
    ''' Get raw spd files and return as dataframe for bus '''
    df = pd.concat((pd.DataFrame(_extract(file, fn, 'SOLUTION,BUS')[1:], 
                                    columns = _extract(file, fn, 'SOLUTION,BUS')[0])
                        for fn in file.namelist() if '.SPDSOLVED' in fn[-10:]), axis=0)
    cols = ['INTERVAL', 'ID_BUS', 'ID_ST', 'LOAD', 'GENERATION']
    return (df
        [cols]
        .assign(INTERVAL = lambda df_: pd.to_datetime(df_.INTERVAL.replace({';':':'}, regex=True),  dayfirst = True),
                ID_BUS = lambda df_: df_.ID_BUS.str.strip(),
                ID_ST = lambda df_: df_.ID_ST.str.strip(),
                LOAD = lambda df_: df_.LOAD.astype('float32'),
                GENERATION = lambda df_: df_.GENERATION.astype('float32')
                ))

def _extract(file, fn, text):
    ''' parse zip file '''
    lines_to_keep = []
    with file.open(fn) as f:
        for line in f.readlines():
            if text in line.decode('utf8'):
                lines_to_keep.append(line.decode('utf8').replace('\n','').replace('\r','').split(','))
    return lines_to_keep

Test operation with one day

In [ ]:
available_days = extract_spd_list(2022)
zip_file = httpx.get(f'https://www.emi.ea.govt.nz/Wholesale/Datasets/FinalPricing/CaseFiles/2022/{available_days.iloc[0].Name}', follow_redirects=True)
file = ZipFile(BytesIO(zip_file.content))
_bus(file)

,INTERVAL,ID_BUS,ID_ST,LOAD,GENERATION
0,2022-01-01 00:00:00,100,MVE,0.000000,0.0000
1,2022-01-01 00:00:00,101,MVE,0.000000,0.0000
2,2022-01-01 00:00:00,102,RTO,0.000000,0.0000
3,2022-01-01 00:00:00,103,CBG,19.943001,0.0000
4,2022-01-01 00:00:00,104,CBG,0.000000,0.0000
...,...,...,...,...,...
939,2022-01-01 23:30:00,95,ARI,0.000000,4.9120
940,2022-01-01 23:30:00,96,ARI,0.000000,15.3333
941,2022-01-01 23:30:00,97,ARI,0.000000,15.3333
942,2022-01-01 23:30:00,98,ARI,0.000000,15.3333


Want to speed up downloads - use httpx async

In [ ]:
#|export
async def get_spd(client, url):
    res = await client.get(url, follow_redirects=True)
    return _branch(ZipFile(BytesIO(res.content))), _bus(ZipFile(BytesIO(res.content)))

async def main_spd(spd_df, year):
    print(year)
    async with httpx.AsyncClient(timeout=None) as client:
        tasks = []
        for file_name in spd_df.Name.to_list():
            url = f'https://www.emi.ea.govt.nz/Wholesale/Datasets/FinalPricing/CaseFiles/{year}/{file_name}'
            tasks.append(asyncio.create_task(get_spd(client, url)))
        res = await asyncio.gather(*tasks)
        return pd.concat([x[0] for x in res]), pd.concat([x[1] for x in res])

Connect and download a month at a time

In [ ]:
month, year = 1 , 2022
spd_df = extract_spd_list(year)
await main_spd(spd_df.loc[f'{year}-{month}'], year)

2022


(                INTERVAL               BRANCHNAME FROM_STATION TO_STATION  \
 0    2022-01-01 00:00:00         ABY     MXT1MXT1          ABY        ABY   
 1    2022-01-01 00:00:00           ABY     T2  T2          ABY        ABY   
 2    2022-01-01 00:00:00  ALB     ALB_HEN3      1          ALB        HEN   
 3    2022-01-01 00:00:00  ALB     ALB_HPI1      1          ALB        HPI   
 4    2022-01-01 00:00:00  ALB     ALB_SVL1      1          ALB        SVL   
 ...                  ...                      ...          ...        ...   
 1077 2022-01-31 23:30:00           WTU     T4  T4          WTU        WTU   
 1078 2022-01-31 23:30:00         WVY     MXT1MXT1          WVY        WVY   
 1079 2022-01-31 23:30:00           WVY     T1  T1          WVY        WVY   
 1080 2022-01-31 23:30:00           WWD     T1  T1          WWD        WWD   
 1081 2022-01-31 23:30:00           WWD     T2  T2          WWD        WWD   
 
         FROM_MW      TO_MW  BRANCHLOSSES  
 0      0.000000  